In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2, MobileNet, NASNetMobile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
batch=16

In [ ]:
# 設置數據生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    validation_split=0.2
)

# 訓練數據生成器
train_generator = train_datagen.flow_from_directory(
    'garbage',  
    target_size=(224, 224),  
    batch_size=batch,
    class_mode='categorical',
    subset='training'  # 指定為訓練集
)

# 驗證數據生成器
validation_generator = train_datagen.flow_from_directory(
    'garbage',
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    subset='validation'  # 指定為驗證集
)

Found 6465 images belonging to 4 classes.
Found 1614 images belonging to 4 classes.


In [ ]:
# 加載預訓練模型
base_model = MobileNetV2(
    alpha=1.3,
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3))

# 添加自定義分類層
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dense(4, activation='softmax')(x)

# 構建完整模型
model = Model(inputs=base_model.input, outputs=x)

# 解凍預訓練層
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[:4]:  
    layer.trainable = True

    
# 編譯模型
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
checkpoint = ModelCheckpoint(
    "1.3MobileNetV2.h5",   # 保存的文件名稱
    monitor="val_accuracy",  # 監測驗證準確率（也可以改成 "val_loss"）
    save_best_only=True,     # 只儲存最佳權重
    mode="max",              # "max" 代表數值越高越好（適用於準確率）
    verbose=1                # 顯示存檔訊息
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    steps_per_epoch=train_generator.samples // batch,
    validation_steps=validation_generator.samples // batch,
    callbacks=[checkpoint]  
)